<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Reading-in-Data" data-toc-modified-id="Reading-in-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Reading in Data</a></span></li><li><span><a href="#Plotting" data-toc-modified-id="Plotting-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Plotting</a></span><ul class="toc-item"><li><span><a href="#Static-Trace" data-toc-modified-id="Static-Trace-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Static Trace</a></span></li></ul></li><li><span><a href="#Polynomial-Fitting" data-toc-modified-id="Polynomial-Fitting-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Polynomial Fitting</a></span></li></ul></div>

From our previous analyses, a majority of our sample seems to be included in Redbull et al. (2016). For this reason we would like to focus on this homogeneous sample. Since they are derived from K2 light curves, they should be more reliable. So we wish to scrutinize this same a but more.

If the object in question is in fact an unresolved binary, the total luminosity will be overestimated, sometimes up to a factor of two--when the secondary star is comparable, in terms of brightness, to the primary, while the color (temperature) remains mostly the same. Such an overestimation would result in an overestimation of Rs. Such systems would appear slightly above the main-sequence in a CMD.

So, we will inspect how plots of Rs-Teff or vsini-v plots change if we removed these points. By doing the following:
1. Fit a low order polynomial to M_G vs BP-RP to define the main sequence
1. Cut the points above the line by some threshold, between 0 and 2.5 log10(2)--which corresponds to a twin binary case.

In [1]:
import pandas as pd
import numpy as np

In [2]:
main_dir = "/u/kaimibk/Documents/Research/NAC/"
data_dir = main_dir+"data/"
out_dir = main_dir+"output/"

## Reading in Data

In [4]:
df = pd.read_csv(data_dir+"Pleiades_master.csv", index_col=0)
df.head()

,Name,RA,Dec,K2MASS,(V-K)o,Period,Ref,BCK,logL/Lo,M/Mo,...,k_msigcom,radius_sigma,lum_sigma,teff_sigma,V_iso,V_iso_err,iso_rad,iso_rad_Err,iso_teff,iso_teff_Err
0,J03394633+2358531,54.943067,23.981442,11.83,4.89,1.138,1,2.73,-1.66,0.41,...,0.022,NaN,NaN,358.076660,18.646667,0.593949,0.419426,0.013360,4222.314643,100.0
1,J03414664+2301198,55.444367,23.022186,13.16,5.63,0.294,1,2.81,-2.23,0.19,...,0.028,NaN,NaN,NaN,46.221644,1.654124,0.268599,0.009612,3951.132391,100.0
2,J03414895+2303235,55.453963,23.056539,13.19,6.09,0.239,1,2.86,-2.26,0.19,...,0.033,NaN,NaN,NaN,57.058377,3.120213,0.269544,0.014740,3875.296173,100.0
3,J03415671+2358434,55.486317,23.978728,13.25,5.76,0.401,1,2.82,-2.27,0.18,...,0.029,NaN,NaN,NaN,38.341851,1.534672,0.303899,0.012164,3788.293001,100.0
4,J03415864+2257020,55.494367,22.950556,11.90,4.78,6.842,1,2.72,-1.68,0.40,...,0.020,NaN,NaN,249.679932,3.021724,0.095273,0.408648,0.012884,4135.851442,100.0


In [55]:
x = df.bp_rp.values
y = (df.phot_g_mean_mag + 5*np.log10(df.parallax*10**(-3) + 1)).values

## Plotting

In [27]:
import plotly.plotly as py
import plotly.graph_objs as go

### Static Trace

In [88]:
data = []

xs = np.arange(0, 4.0, 100)

static = go.Scatter(
    x = x,
    y = y,
    mode = 'markers',
    )
data.append(static)

In [89]:
upper_thresh = 2.5* np.log10(2.0)

## Polynomial Fitting

In [90]:
from scipy.odr import ODR, Data, polynomial

In [91]:
mydata = Data(x,y)
for count, degree in enumerate(range(3,8)):
        #z = np.polyfit(x, y, degree, full=True)
        #p = np.poly1d(z)
        #fit = p(xs)
        model = polynomial(degree)
        myodr = ODR(mydata, model)
        out = myodr.run()
        p = np.poly1d(out.beta)
        fit = p(xs)
        
        lower = go.Scatter(
            mode = "lines",
            showlegend = False,
            
            x = xs,
            y = fit - upper_thresh,
            fill = "tonexty"
            )
        
        trace = go.Scatter(
            mode = "lines",
            name = "Degree: ".format(degree),
            showlegend = False,

            x = xs,
            y = fit,
            fill = "none"
            )
        
        upper = go.Scatter(
            mode = "lines",
            name = "Degree: ".format(degree),
            showlegend = False,

            x = xs,
            y = fit - upper_thresh,
            fill = "tonexty"
            )
        
        data.append(lower)
        data.append(trace)
        data.append(upper)

In [92]:
figure = go.Figure(data=data)

py.iplot(figure, filename="cmd_thresh")